In [1]:
%pip install seaborn


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import seaborn as sns
import os
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from collections import Counter
from pathlib import Path
import textwrap
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('mal_top_anime.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29786 entries, 0 to 29785
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   rank            22428 non-null  float64
 1   mal_id          29786 non-null  int64  
 2   title           29786 non-null  object 
 3   title_english   13124 non-null  object 
 4   title_japanese  29657 non-null  object 
 5   type            29712 non-null  object 
 6   episodes        28962 non-null  float64
 7   status          29786 non-null  object 
 8   aired           29786 non-null  object 
 9   season          6421 non-null   object 
 10  year            6421 non-null   float64
 11  source          29786 non-null  object 
 12  duration        29786 non-null  object 
 13  rating          29178 non-null  object 
 14  score           19545 non-null  float64
 15  scored_by       19545 non-null  float64
 16  members         29786 non-null  int64  
 17  favorites       29786 non-null 

In [2]:


# ─── Config ──────────────────────────────────────────────────────────────────
INPUT_CSV = "mal_top_anime.csv"
OUT_DIR = Path("mal_analysis_report")
OUT_DIR.mkdir(exist_ok=True)

# Style
sns.set_theme(style="whitegrid", font_scale=1.1)
PALETTE = sns.color_palette("viridis", 12)
ACCENT = "#2196F3"
FIG_W, FIG_H = 14, 7

saved_charts = []  # (filename, title, description)


def savefig(fig, name, title="", desc=""):
    path = OUT_DIR / name
    fig.savefig(path, dpi=150, bbox_inches="tight", facecolor="white")
    plt.close(fig)
    saved_charts.append((name, title, desc))
    print(f"  ✅ {name}")


# ═════════════════════════════════════════════════════════════════════════════
#  1. LOAD & CLEAN
# ═════════════════════════════════════════════════════════════════════════════
print("📂 Loading data...")
df = pd.read_csv(INPUT_CSV)
total_rows = len(df)
print(f"   {total_rows:,} anime loaded\n")

# --- Derive extra columns ---
# Extract year from 'aired' when 'year' column is NaN
def extract_year(row):
    if pd.notna(row["year"]):
        return int(row["year"])
    import re
    m = re.search(r"((?:19|20)\d{2})", str(row.get("aired", "")))
    return int(m.group(1)) if m else np.nan

df["year_clean"] = df.apply(extract_year, axis=1)

# Decade
df["decade"] = (df["year_clean"] // 10 * 10).astype("Int64")

# Duration to minutes
def parse_duration(d):
    if pd.isna(d) or d == "Unknown":
        return np.nan
    import re
    hrs = re.search(r"(\d+)\s*hr", str(d))
    mins = re.search(r"(\d+)\s*min", str(d))
    total = 0
    if hrs: total += int(hrs.group(1)) * 60
    if mins: total += int(mins.group(1))
    return total if total > 0 else np.nan

df["duration_min"] = df["duration"].apply(parse_duration)

# Explode genres into individual rows for genre analysis
def explode_col(col_name):
    return (
        df[["mal_id", col_name]]
        .dropna(subset=[col_name])
        .assign(**{col_name: lambda x: x[col_name].str.split(", ")})
        .explode(col_name)
    )

genres_df = explode_col("genres")
themes_df = explode_col("themes")
studios_df = explode_col("studios")

# Filtered dataframe for scored anime
scored = df[df["score"].notna()].copy()
ranked = df[df["rank"].notna()].copy()

print("🔍 Starting analysis...\n")

# ═════════════════════════════════════════════════════════════════════════════
#  2. DATASET OVERVIEW
# ═════════════════════════════════════════════════════════════════════════════
print("─── Section 1: Dataset Overview ───")

overview_stats = {
    "Total Anime": f"{total_rows:,}",
    "With Scores": f"{len(scored):,} ({len(scored)/total_rows*100:.1f}%)",
    "With Rankings": f"{len(ranked):,}",
    "Unique Types": df["type"].nunique(),
    "Year Range": f"{int(df['year_clean'].min())}-{int(df['year_clean'].max())}" if df["year_clean"].notna().any() else "N/A",
    "Score Range": f"{scored['score'].min():.2f} — {scored['score'].max():.2f}",
    "Avg Score": f"{scored['score'].mean():.2f}",
    "Median Score": f"{scored['score'].median():.2f}",
    "Total Members (sum)": f"{df['members'].sum():,.0f}",
    "Unique Genres": genres_df["genres"].nunique(),
    "Unique Studios": studios_df["studios"].nunique(),
}

# ── Chart 1: Missing data heatmap ──
fig, ax = plt.subplots(figsize=(FIG_W, 5))
missing = df.isnull().mean().sort_values(ascending=False) * 100
missing_cols = missing[missing > 0]
bars = ax.barh(missing_cols.index, missing_cols.values, color=PALETTE[2])
ax.set_xlabel("% Missing")
ax.set_title("Missing Data by Column", fontsize=16, fontweight="bold")
for bar, val in zip(bars, missing_cols.values):
    ax.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2,
            f"{val:.1f}%", va="center", fontsize=10)
ax.invert_yaxis()
fig.tight_layout()
savefig(fig, "01_missing_data.png", "Missing Data Overview",
        f"{len(missing_cols)} columns have missing values. 'season' and 'year' are most sparse (only available for ~6.4K entries).")


# ═════════════════════════════════════════════════════════════════════════════
#  3. SCORE ANALYSIS
# ═════════════════════════════════════════════════════════════════════════════
print("─── Section 2: Score Analysis ───")

# ── Chart 2: Score distribution ──
fig, axes = plt.subplots(1, 2, figsize=(FIG_W, FIG_H))

axes[0].hist(scored["score"], bins=60, color=ACCENT, edgecolor="white", alpha=0.85)
axes[0].axvline(scored["score"].mean(), color="red", ls="--", lw=2, label=f'Mean: {scored["score"].mean():.2f}')
axes[0].axvline(scored["score"].median(), color="orange", ls="--", lw=2, label=f'Median: {scored["score"].median():.2f}')
axes[0].set_xlabel("Score")
axes[0].set_ylabel("Count")
axes[0].set_title("Score Distribution", fontweight="bold")
axes[0].legend()

# Box plot by type
type_order = scored.groupby("type")["score"].median().sort_values(ascending=False).index
sns.boxplot(data=scored, y="type", x="score", order=type_order, ax=axes[1], palette="viridis")
axes[1].set_title("Score Distribution by Type", fontweight="bold")
axes[1].set_xlabel("Score")
axes[1].set_ylabel("")
fig.tight_layout()
savefig(fig, "02_score_distribution.png", "Score Distribution",
        f"Mean score: {scored['score'].mean():.2f}, Median: {scored['score'].median():.2f}. Scores are left-skewed, clustering around 6-7.")

# ── Chart 3: Score percentiles ──
fig, ax = plt.subplots(figsize=(FIG_W, 5))
percentiles = [1, 5, 10, 25, 50, 75, 90, 95, 99]
pct_values = [scored["score"].quantile(p/100) for p in percentiles]
bars = ax.bar([f"P{p}" for p in percentiles], pct_values, color=sns.color_palette("coolwarm", len(percentiles)))
for bar, val in zip(bars, pct_values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05, f"{val:.2f}",
            ha="center", fontweight="bold", fontsize=11)
ax.set_ylabel("Score")
ax.set_title("Score Percentiles", fontsize=16, fontweight="bold")
ax.set_ylim(0, 10)
fig.tight_layout()
savefig(fig, "03_score_percentiles.png", "Score Percentiles",
        f"Top 1% of anime have a score ≥ {scored['score'].quantile(0.99):.2f}. Bottom 1% score ≤ {scored['score'].quantile(0.01):.2f}.")


# ═════════════════════════════════════════════════════════════════════════════
#  4. TYPE BREAKDOWN
# ═════════════════════════════════════════════════════════════════════════════
print("─── Section 3: Type Breakdown ───")

fig, axes = plt.subplots(1, 2, figsize=(FIG_W, FIG_H))

type_counts = df["type"].value_counts()
colors_pie = sns.color_palette("Set2", len(type_counts))
wedges, texts, autotexts = axes[0].pie(
    type_counts.values, labels=type_counts.index, autopct="%1.1f%%",
    colors=colors_pie, startangle=90, pctdistance=0.8)
axes[0].set_title("Anime Count by Type", fontweight="bold")

# Avg score by type
type_score = scored.groupby("type")["score"].agg(["mean", "count"]).sort_values("mean", ascending=True)
bars = axes[1].barh(type_score.index, type_score["mean"], color=PALETTE[4])
for bar, (idx, row) in zip(bars, type_score.iterrows()):
    axes[1].text(bar.get_width() + 0.03, bar.get_y() + bar.get_height()/2,
                 f'{row["mean"]:.2f} (n={int(row["count"]):,})', va="center", fontsize=10)
axes[1].set_xlabel("Average Score")
axes[1].set_title("Average Score by Type", fontweight="bold")
axes[1].set_xlim(0, 10)
fig.tight_layout()
savefig(fig, "04_type_breakdown.png", "Type Breakdown",
        f"Most common type: {type_counts.index[0]} ({type_counts.values[0]:,}). Highest avg score type: {type_score.index[-1]}.")


# ═════════════════════════════════════════════════════════════════════════════
#  5. TEMPORAL TRENDS
# ═════════════════════════════════════════════════════════════════════════════
print("─── Section 4: Temporal Trends ───")

yearly = df[df["year_clean"].notna()].copy()
yearly["year_clean"] = yearly["year_clean"].astype(int)

# ── Chart 5: Anime produced per year ──
fig, axes = plt.subplots(2, 1, figsize=(FIG_W, 10), sharex=True)

year_counts = yearly.groupby("year_clean").size()
axes[0].fill_between(year_counts.index, year_counts.values, alpha=0.3, color=ACCENT)
axes[0].plot(year_counts.index, year_counts.values, color=ACCENT, lw=2)
axes[0].set_ylabel("Number of Anime")
axes[0].set_title("Anime Produced Per Year", fontsize=16, fontweight="bold")
axes[0].annotate(f"Peak: {year_counts.idxmax()} ({year_counts.max():,})",
                 xy=(year_counts.idxmax(), year_counts.max()),
                 xytext=(year_counts.idxmax() - 10, year_counts.max() * 0.85),
                 arrowprops=dict(arrowstyle="->", color="red"), fontsize=12, color="red")

# Avg score per year
year_score = yearly[yearly["score"].notna()].groupby("year_clean")["score"].mean()
axes[1].plot(year_score.index, year_score.values, color="#E91E63", lw=2)
axes[1].fill_between(year_score.index, year_score.values, alpha=0.2, color="#E91E63")
axes[1].set_ylabel("Average Score")
axes[1].set_xlabel("Year")
axes[1].set_title("Average Score Over Time", fontsize=14, fontweight="bold")
axes[1].set_ylim(4, 8)
fig.tight_layout()
savefig(fig, "05_temporal_trends.png", "Temporal Trends",
        f"Anime production peaked in {year_counts.idxmax()} with {year_counts.max():,} titles.")

# ── Chart 6: Decade comparison ──
decade_data = yearly[yearly["decade"].notna() & (yearly["decade"] >= 1960)]
fig, axes = plt.subplots(1, 2, figsize=(FIG_W, FIG_H))

decade_counts = decade_data.groupby("decade").size()
axes[0].bar(decade_counts.index.astype(str), decade_counts.values, color=sns.color_palette("rocket", len(decade_counts)))
axes[0].set_title("Anime Count by Decade", fontweight="bold")
axes[0].set_ylabel("Count")
for i, (d, c) in enumerate(zip(decade_counts.index, decade_counts.values)):
    axes[0].text(i, c + 50, f"{c:,}", ha="center", fontsize=10, fontweight="bold")

# Score distribution by decade
decade_scored = decade_data[decade_data["score"].notna() & (decade_data["decade"] >= 1970)]
decade_order = sorted(decade_scored["decade"].unique())
sns.violinplot(data=decade_scored, x="decade", y="score", order=decade_order,
               ax=axes[1], palette="rocket", inner="quartile")
axes[1].set_title("Score Distribution by Decade", fontweight="bold")
axes[1].set_xlabel("Decade")
axes[1].set_ylabel("Score")
fig.tight_layout()
savefig(fig, "06_decade_analysis.png", "Decade Analysis",
        "The 2010s and 2020s saw an explosion of anime production.")

# ── Chart 7: Seasonal patterns ──
seasonal = df[df["season"].notna()].copy()
if len(seasonal) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(FIG_W, FIG_H))
    season_order = ["winter", "spring", "summer", "fall"]
    season_colors = {"winter": "#42A5F5", "spring": "#66BB6A", "summer": "#FFA726", "fall": "#EF5350"}

    season_counts = seasonal["season"].value_counts().reindex(season_order)
    axes[0].bar(season_counts.index, season_counts.values,
                color=[season_colors.get(s, "gray") for s in season_counts.index])
    axes[0].set_title("Anime Count by Season", fontweight="bold")
    axes[0].set_ylabel("Count")

    season_score = seasonal[seasonal["score"].notna()].groupby("season")["score"].mean().reindex(season_order)
    axes[1].bar(season_score.index, season_score.values,
                color=[season_colors.get(s, "gray") for s in season_score.index])
    axes[1].set_title("Average Score by Season", fontweight="bold")
    axes[1].set_ylabel("Average Score")
    axes[1].set_ylim(5, 8)
    for i, (s, v) in enumerate(season_score.items()):
        axes[1].text(i, v + 0.05, f"{v:.2f}", ha="center", fontweight="bold")
    fig.tight_layout()
    savefig(fig, "07_seasonal_patterns.png", "Seasonal Patterns",
            "Analysis of anime releases by season (winter, spring, summer, fall).")


# ═════════════════════════════════════════════════════════════════════════════
#  6. GENRE ANALYSIS
# ═════════════════════════════════════════════════════════════════════════════
print("─── Section 5: Genre Analysis ───")

genre_counts = genres_df["genres"].value_counts()

# ── Chart 8: Top genres ──
fig, axes = plt.subplots(1, 2, figsize=(FIG_W, 8))
top_n = 20
top_genres = genre_counts.head(top_n)
bars = axes[0].barh(range(top_n), top_genres.values, color=sns.color_palette("viridis", top_n))
axes[0].set_yticks(range(top_n))
axes[0].set_yticklabels(top_genres.index)
axes[0].invert_yaxis()
axes[0].set_xlabel("Count")
axes[0].set_title(f"Top {top_n} Genres by Count", fontweight="bold")
for bar, val in zip(bars, top_genres.values):
    axes[0].text(bar.get_width() + 50, bar.get_y() + bar.get_height()/2,
                 f"{val:,}", va="center", fontsize=9)

# Avg score by genre (top 20 genres only)
genre_score = (
    genres_df.merge(scored[["mal_id", "score"]], on="mal_id")
    .groupby("genres")["score"]
    .agg(["mean", "count"])
    .query("count >= 50")
    .sort_values("mean", ascending=True)
    .tail(20)
)
bars2 = axes[1].barh(genre_score.index, genre_score["mean"], color=sns.color_palette("magma", 20))
axes[1].set_xlabel("Average Score")
axes[1].set_title("Top 20 Genres by Avg Score (min 50 entries)", fontweight="bold")
axes[1].set_xlim(5, 8)
for bar, (idx, row) in zip(bars2, genre_score.iterrows()):
    axes[1].text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2,
                 f'{row["mean"]:.2f}', va="center", fontsize=9)
fig.tight_layout()
savefig(fig, "08_genre_analysis.png", "Genre Analysis",
        f"Most common genre: {genre_counts.index[0]} ({genre_counts.values[0]:,}). Highest rated genre: {genre_score.index[-1]}.")

# ── Chart 9: Genre co-occurrence heatmap ──
top_10_genres = genre_counts.head(12).index.tolist()
co_matrix = pd.DataFrame(0, index=top_10_genres, columns=top_10_genres)
anime_genres = df.dropna(subset=["genres"]).set_index("mal_id")["genres"].str.split(", ")
for glist in anime_genres:
    filtered = [g for g in glist if g in top_10_genres]
    for i, g1 in enumerate(filtered):
        for g2 in filtered[i+1:]:
            co_matrix.loc[g1, g2] += 1
            co_matrix.loc[g2, g1] += 1

fig, ax = plt.subplots(figsize=(10, 8))
mask = np.triu(np.ones_like(co_matrix, dtype=bool), k=0)
sns.heatmap(co_matrix, mask=mask, annot=True, fmt="d", cmap="YlOrRd",
            ax=ax, linewidths=0.5, square=True)
ax.set_title("Genre Co-occurrence (Top 12 Genres)", fontsize=16, fontweight="bold")
fig.tight_layout()
savefig(fig, "09_genre_cooccurrence.png", "Genre Co-occurrence Heatmap",
        "Shows how often the top genres appear together on the same anime.")

# ── Chart 10: Genre trends over decades ──
genre_year = (
    genres_df.merge(yearly[["mal_id", "decade"]], on="mal_id")
    .query("decade >= 1980")
)
top_6_genres = genre_counts.head(6).index.tolist()
genre_trend = (
    genre_year[genre_year["genres"].isin(top_6_genres)]
    .groupby(["decade", "genres"]).size()
    .unstack(fill_value=0)
)
fig, ax = plt.subplots(figsize=(FIG_W, FIG_H))
genre_trend.plot(kind="bar", stacked=False, ax=ax, colormap="Set2", width=0.8)
ax.set_title("Top 6 Genre Trends by Decade", fontsize=16, fontweight="bold")
ax.set_xlabel("Decade")
ax.set_ylabel("Count")
ax.legend(title="Genre", bbox_to_anchor=(1.02, 1), loc="upper left")
ax.set_xticklabels([str(int(x)) for x in genre_trend.index], rotation=0)
fig.tight_layout()
savefig(fig, "10_genre_trends.png", "Genre Trends Over Decades",
        "How the popularity of top genres has shifted across decades.")


# ═════════════════════════════════════════════════════════════════════════════
#  7. STUDIO ANALYSIS
# ═════════════════════════════════════════════════════════════════════════════
print("─── Section 6: Studio Analysis ───")

studio_counts = studios_df["studios"].value_counts()
studio_with_score = studios_df.merge(scored[["mal_id", "score", "members"]], on="mal_id")

# ── Chart 11: Top studios ──
fig, axes = plt.subplots(1, 2, figsize=(FIG_W, 9))
top_20_studios = studio_counts.head(20)
axes[0].barh(range(20), top_20_studios.values, color=sns.color_palette("mako", 20))
axes[0].set_yticks(range(20))
axes[0].set_yticklabels(top_20_studios.index)
axes[0].invert_yaxis()
axes[0].set_xlabel("Number of Anime")
axes[0].set_title("Top 20 Studios by Output", fontweight="bold")

studio_quality = (
    studio_with_score.groupby("studios")["score"]
    .agg(["mean", "count"])
    .query("count >= 20")
    .sort_values("mean", ascending=True)
    .tail(20)
)
axes[1].barh(studio_quality.index, studio_quality["mean"], color=sns.color_palette("mako_r", 20))
axes[1].set_xlabel("Average Score")
axes[1].set_title("Top 20 Studios by Avg Score (min 20 anime)", fontweight="bold")
axes[1].set_xlim(5.5, 8.5)
for i, (idx, row) in enumerate(studio_quality.iterrows()):
    axes[1].text(row["mean"] + 0.02, i, f'{row["mean"]:.2f} (n={int(row["count"])})', va="center", fontsize=9)
fig.tight_layout()
savefig(fig, "11_studio_analysis.png", "Studio Analysis",
        f"Most prolific studio: {studio_counts.index[0]} ({studio_counts.values[0]:,} anime). Highest rated: {studio_quality.index[-1]}.")


# ═════════════════════════════════════════════════════════════════════════════
#  8. POPULARITY vs QUALITY
# ═════════════════════════════════════════════════════════════════════════════
print("─── Section 7: Popularity vs Quality ───")

pop_df = scored[scored["members"] > 0].copy()
pop_df["log_members"] = np.log10(pop_df["members"])

fig, axes = plt.subplots(1, 2, figsize=(FIG_W, FIG_H))

# Scatter: members vs score
axes[0].scatter(pop_df["log_members"], pop_df["score"], alpha=0.15, s=8, c=ACCENT)
z = np.polyfit(pop_df["log_members"], pop_df["score"], 1)
p = np.poly1d(z)
x_line = np.linspace(pop_df["log_members"].min(), pop_df["log_members"].max(), 100)
axes[0].plot(x_line, p(x_line), "r--", lw=2, label=f"Trend (slope={z[0]:.2f})")
axes[0].set_xlabel("Members (log₁₀)")
axes[0].set_ylabel("Score")
axes[0].set_title("Popularity vs Score", fontweight="bold")
axes[0].legend()
corr = pop_df["log_members"].corr(pop_df["score"])
axes[0].text(0.05, 0.95, f"r = {corr:.3f}", transform=axes[0].transAxes,
             fontsize=14, verticalalignment="top",
             bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.5))

# Scatter: scored_by vs score
fav_df = scored[scored["favorites"] > 0].copy()
fav_df["log_fav"] = np.log10(fav_df["favorites"])
axes[1].scatter(fav_df["log_fav"], fav_df["score"], alpha=0.15, s=8, c="#E91E63")
corr2 = fav_df["log_fav"].corr(fav_df["score"])
axes[1].set_xlabel("Favorites (log₁₀)")
axes[1].set_ylabel("Score")
axes[1].set_title("Favorites vs Score", fontweight="bold")
axes[1].text(0.05, 0.95, f"r = {corr2:.3f}", transform=axes[1].transAxes,
             fontsize=14, verticalalignment="top",
             bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.5))
fig.tight_layout()
savefig(fig, "12_popularity_vs_quality.png", "Popularity vs Quality",
        f"Correlation between log(members) and score: r={corr:.3f}. More popular anime tend to have higher scores.")

# ── Chart 13: Most popular anime (top 30 by members) ──
top_popular = df.nlargest(30, "members")[["title", "members", "score", "type"]].reset_index(drop=True)
fig, ax = plt.subplots(figsize=(FIG_W, 9))
colors_bar = ["#FF6B6B" if s and s >= 8.5 else "#4ECDC4" if s and s >= 7 else "#95E1D3"
              for s in top_popular["score"]]
bars = ax.barh(range(30), top_popular["members"], color=colors_bar)
ax.set_yticks(range(30))
ax.set_yticklabels([textwrap.shorten(t, width=40, placeholder="...") for t in top_popular["title"]], fontsize=9)
ax.invert_yaxis()
ax.set_xlabel("Members")
ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{x/1e6:.1f}M"))
ax.set_title("Top 30 Most Popular Anime (by Members)", fontsize=16, fontweight="bold")
fig.tight_layout()
savefig(fig, "13_most_popular.png", "Most Popular Anime",
        f"Most popular: {top_popular.iloc[0]['title']} ({top_popular.iloc[0]['members']:,} members).")


# ═════════════════════════════════════════════════════════════════════════════
#  9. EPISODES & DURATION
# ═════════════════════════════════════════════════════════════════════════════
print("─── Section 8: Episodes & Duration ───")

eps_df = scored[scored["episodes"].notna() & (scored["episodes"] > 0)].copy()

fig, axes = plt.subplots(1, 2, figsize=(FIG_W, FIG_H))

# Episode count distribution (cap at 100 for readability)
eps_capped = eps_df["episodes"].clip(upper=100)
axes[0].hist(eps_capped, bins=50, color=ACCENT, edgecolor="white")
axes[0].set_xlabel("Episodes (capped at 100)")
axes[0].set_ylabel("Count")
axes[0].set_title("Episode Count Distribution", fontweight="bold")
axes[0].axvline(eps_df["episodes"].median(), color="red", ls="--", lw=2,
                label=f'Median: {eps_df["episodes"].median():.0f}')
axes[0].legend()

# Score vs episodes (binned)
bins = [0, 1, 6, 13, 26, 52, 100, 500, 10000]
labels = ["1", "2-6", "7-13", "14-26", "27-52", "53-100", "101-500", "500+"]
eps_df["ep_bin"] = pd.cut(eps_df["episodes"], bins=bins, labels=labels)
ep_score = eps_df.groupby("ep_bin", observed=True)["score"].agg(["mean", "count"])
axes[1].bar(ep_score.index.astype(str), ep_score["mean"], color=sns.color_palette("coolwarm", len(ep_score)))
axes[1].set_xlabel("Episode Range")
axes[1].set_ylabel("Average Score")
axes[1].set_title("Score by Episode Count Range", fontweight="bold")
axes[1].set_ylim(5, 8)
for i, (idx, row) in enumerate(ep_score.iterrows()):
    axes[1].text(i, row["mean"] + 0.05, f'{row["mean"]:.2f}\n(n={int(row["count"])})',
                 ha="center", fontsize=9)
fig.tight_layout()
savefig(fig, "14_episodes_duration.png", "Episodes & Duration Analysis",
        f"Median episode count: {eps_df['episodes'].median():.0f}.")


# ═════════════════════════════════════════════════════════════════════════════
#  10. SOURCE MATERIAL
# ═════════════════════════════════════════════════════════════════════════════
print("─── Section 9: Source Material ───")

fig, axes = plt.subplots(1, 2, figsize=(FIG_W, FIG_H))

source_counts = df["source"].value_counts().head(10)
axes[0].barh(source_counts.index[::-1], source_counts.values[::-1],
             color=sns.color_palette("Set2", 10))
axes[0].set_xlabel("Count")
axes[0].set_title("Top 10 Source Materials", fontweight="bold")

source_score = (
    scored.groupby("source")["score"].agg(["mean", "count"])
    .query("count >= 30")
    .sort_values("mean", ascending=True)
)
axes[1].barh(source_score.index, source_score["mean"],
             color=sns.color_palette("Set2", len(source_score)))
axes[1].set_xlabel("Average Score")
axes[1].set_title("Avg Score by Source (min 30 entries)", fontweight="bold")
axes[1].set_xlim(5, 8)
fig.tight_layout()
savefig(fig, "15_source_material.png", "Source Material Analysis",
        f"Most common source: {source_counts.index[0]} ({source_counts.values[0]:,}).")


# ═════════════════════════════════════════════════════════════════════════════
#  11. RATING (AGE) ANALYSIS
# ═════════════════════════════════════════════════════════════════════════════
print("─── Section 10: Rating (Age) Analysis ───")

rating_df = scored[scored["rating"].notna()]

fig, axes = plt.subplots(1, 2, figsize=(FIG_W, FIG_H))

rating_counts = rating_df["rating"].value_counts()
axes[0].barh(rating_counts.index[::-1], rating_counts.values[::-1],
             color=sns.color_palette("Spectral", len(rating_counts)))
axes[0].set_xlabel("Count")
axes[0].set_title("Anime by Age Rating", fontweight="bold")

rating_order = rating_df.groupby("rating")["score"].median().sort_values().index
sns.boxplot(data=rating_df, y="rating", x="score", order=rating_order,
            ax=axes[1], palette="Spectral")
axes[1].set_title("Score by Age Rating", fontweight="bold")
axes[1].set_xlabel("Score")
fig.tight_layout()
savefig(fig, "16_rating_analysis.png", "Age Rating Analysis",
        "Score distributions vary by content rating.")


# ═════════════════════════════════════════════════════════════════════════════
#  12. TOP & BOTTOM ANIME
# ═════════════════════════════════════════════════════════════════════════════
print("─── Section 11: Top & Bottom Anime ───")

# Top 25
top25 = scored.nlargest(25, "score")[["rank", "title", "score", "type", "members"]].reset_index(drop=True)
fig, ax = plt.subplots(figsize=(FIG_W, 8))
bars = ax.barh(range(25), top25["score"], color=sns.color_palette("YlOrRd_r", 25))
ax.set_yticks(range(25))
ax.set_yticklabels([textwrap.shorten(t, width=45, placeholder="...") for t in top25["title"]], fontsize=9)
ax.invert_yaxis()
ax.set_xlabel("Score")
ax.set_xlim(8.5, top25["score"].max() + 0.15)
ax.set_title("Top 25 Highest Rated Anime", fontsize=16, fontweight="bold")
for bar, val in zip(bars, top25["score"]):
    ax.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2,
            f"{val:.2f}", va="center", fontsize=10, fontweight="bold")
fig.tight_layout()
savefig(fig, "17_top25_anime.png", "Top 25 Anime",
        f"#1: {top25.iloc[0]['title']} ({top25.iloc[0]['score']:.2f}).")

# Bottom 25
bottom25 = scored.nsmallest(25, "score")[["rank", "title", "score", "type", "members"]].reset_index(drop=True)
fig, ax = plt.subplots(figsize=(FIG_W, 8))
bars = ax.barh(range(25), bottom25["score"], color=sns.color_palette("Reds", 25))
ax.set_yticks(range(25))
ax.set_yticklabels([textwrap.shorten(t, width=45, placeholder="...") for t in bottom25["title"]], fontsize=9)
ax.invert_yaxis()
ax.set_xlabel("Score")
ax.set_title("Bottom 25 Lowest Rated Anime", fontsize=16, fontweight="bold")
for bar, val in zip(bars, bottom25["score"]):
    ax.text(bar.get_width() + 0.02, bar.get_y() + bar.get_height()/2,
            f"{val:.2f}", va="center", fontsize=10, fontweight="bold")
fig.tight_layout()
savefig(fig, "18_bottom25_anime.png", "Bottom 25 Anime",
        f"Lowest rated: {bottom25.iloc[0]['title']} ({bottom25.iloc[0]['score']:.2f}).")


# ═════════════════════════════════════════════════════════════════════════════
#  13. HIDDEN GEMS & OVERRATED
# ═════════════════════════════════════════════════════════════════════════════
print("─── Section 12: Hidden Gems & Overrated ───")

# Hidden gems: high score, low members
gems_df = scored[(scored["score"] >= 8.0) & (scored["members"] < scored["members"].median())].copy()
gems_df = gems_df.nlargest(20, "score")

# Overrated: high members, mediocre score
overrated = scored[(scored["members"] > scored["members"].quantile(0.9)) & (scored["score"] < 7.0)].copy()
overrated = overrated.nsmallest(20, "score")

fig, axes = plt.subplots(1, 2, figsize=(FIG_W, 8))

if len(gems_df) > 0:
    axes[0].barh(range(len(gems_df)), gems_df["score"].values, color="#2ECC71")
    axes[0].set_yticks(range(len(gems_df)))
    axes[0].set_yticklabels([textwrap.shorten(t, 35, placeholder="...") for t in gems_df["title"]], fontsize=8)
    axes[0].invert_yaxis()
    axes[0].set_xlabel("Score")
    axes[0].set_title("🌟 Hidden Gems (High Score, Low Popularity)", fontweight="bold", fontsize=12)
    axes[0].set_xlim(7.5, gems_df["score"].max() + 0.2)

if len(overrated) > 0:
    axes[1].barh(range(len(overrated)), overrated["score"].values, color="#E74C3C")
    axes[1].set_yticks(range(len(overrated)))
    axes[1].set_yticklabels([textwrap.shorten(t, 35, placeholder="...") for t in overrated["title"]], fontsize=8)
    axes[1].invert_yaxis()
    axes[1].set_xlabel("Score")
    axes[1].set_title("📉 Most Popular, Lowest Rated (Top 10% Members)", fontweight="bold", fontsize=12)

fig.tight_layout()
savefig(fig, "19_hidden_gems.png", "Hidden Gems vs Overrated",
        "Left: High quality, low popularity. Right: Very popular but lower scores.")


# ═════════════════════════════════════════════════════════════════════════════
#  14. THEMES ANALYSIS
# ═════════════════════════════════════════════════════════════════════════════
print("─── Section 13: Themes ───")

theme_counts = themes_df["themes"].value_counts().head(20)
theme_scores = (
    themes_df.merge(scored[["mal_id", "score"]], on="mal_id")
    .groupby("themes")["score"].agg(["mean", "count"])
    .query("count >= 30")
    .sort_values("mean", ascending=False)
    .head(20)
)

fig, axes = plt.subplots(1, 2, figsize=(FIG_W, 8))
axes[0].barh(theme_counts.index[::-1], theme_counts.values[::-1],
             color=sns.color_palette("husl", 20))
axes[0].set_xlabel("Count")
axes[0].set_title("Top 20 Themes", fontweight="bold")

axes[1].barh(theme_scores.index[::-1], theme_scores["mean"][::-1],
             color=sns.color_palette("husl", 20))
axes[1].set_xlabel("Average Score")
axes[1].set_title("Top 20 Themes by Avg Score (min 30)", fontweight="bold")
axes[1].set_xlim(5.5, 8)
fig.tight_layout()
savefig(fig, "20_themes_analysis.png", "Themes Analysis",
        f"Most common theme: {theme_counts.index[0]} ({theme_counts.values[0]:,}).")


# ═════════════════════════════════════════════════════════════════════════════
#  15. CORRELATION HEATMAP
# ═════════════════════════════════════════════════════════════════════════════
print("─── Section 14: Correlations ───")

numeric_cols = ["score", "scored_by", "members", "favorites", "episodes", "duration_min"]
corr_matrix = scored[numeric_cols].corr()

fig, ax = plt.subplots(figsize=(9, 7))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="RdBu_r", center=0,
            square=True, ax=ax, linewidths=0.5,
            cbar_kws={"shrink": 0.8})
ax.set_title("Correlation Matrix (Numerical Features)", fontsize=16, fontweight="bold")
fig.tight_layout()
savefig(fig, "21_correlations.png", "Correlation Matrix",
        "Members, scored_by, and favorites are highly correlated. Score shows moderate positive correlation with popularity metrics.")


# ═════════════════════════════════════════════════════════════════════════════
#  16. STATUS BREAKDOWN
# ═════════════════════════════════════════════════════════════════════════════
print("─── Section 15: Status ───")

fig, axes = plt.subplots(1, 2, figsize=(FIG_W, FIG_H))
status_counts = df["status"].value_counts()
axes[0].pie(status_counts.values, labels=status_counts.index, autopct="%1.1f%%",
            colors=sns.color_palette("pastel"), startangle=90)
axes[0].set_title("Anime by Status", fontweight="bold")

status_score = scored.groupby("status")["score"].mean().sort_values()
axes[1].barh(status_score.index, status_score.values, color=sns.color_palette("pastel", len(status_score)))
axes[1].set_xlabel("Average Score")
axes[1].set_title("Average Score by Status", fontweight="bold")
axes[1].set_xlim(5, 8)
fig.tight_layout()
savefig(fig, "22_status_breakdown.png", "Status Breakdown",
        "Distribution of anime by airing status.")


# ═════════════════════════════════════════════════════════════════════════════
#  17. WORDCLOUD FROM SYNOPSIS
# ═════════════════════════════════════════════════════════════════════════════
print("─── Section 16: Synopsis Wordcloud ───")

try:
    from wordcloud import WordCloud
    synopsis_text = " ".join(df["synopsis"].dropna().astype(str).tolist())
    wc = WordCloud(width=1400, height=700, background_color="white",
                   colormap="viridis", max_words=150,
                   stopwords={"the", "and", "to", "of", "a", "in", "is", "with",
                              "his", "her", "he", "she", "who", "that", "it",
                              "for", "on", "was", "are", "but", "they", "their",
                              "from", "an", "as", "by", "has", "been", "will",
                              "one", "them", "this", "all", "not", "when", "at",
                              "no", "can", "be", "had", "into", "after", "about",
                              "its", "which", "more", "than", "out", "up", "him",
                              "do", "what", "where", "how", "each", "other", "also",
                              "Source", "MAL", "Rewrite"}).generate(synopsis_text)

    fig, ax = plt.subplots(figsize=(FIG_W, 7))
    ax.imshow(wc, interpolation="bilinear")
    ax.axis("off")
    ax.set_title("Most Common Words in Anime Synopses", fontsize=16, fontweight="bold")
    fig.tight_layout()
    savefig(fig, "23_synopsis_wordcloud.png", "Synopsis Word Cloud",
            "Common themes in anime descriptions.")
except ImportError:
    print("  ⚠ wordcloud not installed, skipping.")


# ═════════════════════════════════════════════════════════════════════════════
#  18. GENERATE HTML REPORT
# ═════════════════════════════════════════════════════════════════════════════
print("\n─── Generating HTML Report ───")

html_parts = ["""<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>MyAnimeList Analysis Report</title>
<style>
  * { margin: 0; padding: 0; box-sizing: border-box; }
  body { font-family: 'Segoe UI', system-ui, -apple-system, sans-serif;
         background: #0f0f23; color: #e0e0e0; line-height: 1.6; }
  .header { background: linear-gradient(135deg, #1a1a3e 0%, #2d2d6b 50%, #1a1a3e 100%);
            padding: 60px 20px; text-align: center; border-bottom: 3px solid #4a4aff; }
  .header h1 { font-size: 2.8em; color: #fff; margin-bottom: 10px;
               text-shadow: 0 0 20px rgba(74,74,255,0.5); }
  .header p { font-size: 1.2em; color: #aaa; }
  .container { max-width: 1200px; margin: 0 auto; padding: 30px 20px; }
  .stats-grid { display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
                gap: 15px; margin: 30px 0; }
  .stat-card { background: #1a1a3e; border-radius: 12px; padding: 20px;
               text-align: center; border: 1px solid #2d2d6b;
               transition: transform 0.2s; }
  .stat-card:hover { transform: translateY(-3px); }
  .stat-card .value { font-size: 1.8em; font-weight: bold; color: #4a9eff; }
  .stat-card .label { font-size: 0.9em; color: #888; margin-top: 5px; }
  .chart-section { background: #1a1a3e; border-radius: 12px; margin: 30px 0;
                   padding: 25px; border: 1px solid #2d2d6b; }
  .chart-section h2 { color: #4a9eff; margin-bottom: 10px; font-size: 1.5em; }
  .chart-section p { color: #999; margin-bottom: 15px; font-size: 0.95em; }
  .chart-section img { width: 100%; border-radius: 8px; background: white; }
  .footer { text-align: center; padding: 40px; color: #555; }
  .toc { background: #1a1a3e; border-radius: 12px; padding: 25px;
         margin: 30px 0; border: 1px solid #2d2d6b; }
  .toc h2 { color: #4a9eff; margin-bottom: 15px; }
  .toc a { color: #7a7aff; text-decoration: none; display: block; padding: 4px 0; }
  .toc a:hover { color: #aaaaff; }
</style>
</head>
<body>
<div class="header">
  <h1>📊 MyAnimeList Analysis Report</h1>
  <p>Comprehensive analysis of """ + f'{total_rows:,}' + """ anime from MAL's top ranked list</p>
</div>
<div class="container">
"""]

# Stats grid
html_parts.append('<div class="stats-grid">')
for label, value in overview_stats.items():
    html_parts.append(f'<div class="stat-card"><div class="value">{value}</div><div class="label">{label}</div></div>')
html_parts.append('</div>')

# TOC
html_parts.append('<div class="toc"><h2>📋 Table of Contents</h2>')
for fname, title, _ in saved_charts:
    anchor = fname.replace(".png", "")
    html_parts.append(f'<a href="#{anchor}">• {title}</a>')
html_parts.append('</div>')

# Charts
for fname, title, desc in saved_charts:
    anchor = fname.replace(".png", "")
    html_parts.append(f"""
    <div class="chart-section" id="{anchor}">
      <h2>{title}</h2>
      <p>{desc}</p>
      <img src="{fname}" alt="{title}" loading="lazy">
    </div>
    """)

html_parts.append("""
</div>
<div class="footer">
  <p>Generated with Python · pandas · matplotlib · seaborn</p>
</div>
</body></html>""")

report_path = OUT_DIR / "report.html"
with open(report_path, "w", encoding="utf-8") as f:
    f.write("\n".join(html_parts))

print(f"\n{'═' * 60}")
print(f"  ✅ ANALYSIS COMPLETE!")
print(f"  📊 {len(saved_charts)} charts generated")
print(f"  📁 Output folder: {OUT_DIR.resolve()}")
print(f"  🌐 Open report:   {report_path.resolve()}")
print(f"{'═' * 60}")

📂 Loading data...
   29,786 anime loaded

🔍 Starting analysis...

─── Section 1: Dataset Overview ───
  ✅ 01_missing_data.png
─── Section 2: Score Analysis ───
  ✅ 02_score_distribution.png
  ✅ 03_score_percentiles.png
─── Section 3: Type Breakdown ───
  ✅ 04_type_breakdown.png
─── Section 4: Temporal Trends ───
  ✅ 05_temporal_trends.png
  ✅ 06_decade_analysis.png
  ✅ 07_seasonal_patterns.png
─── Section 5: Genre Analysis ───
  ✅ 08_genre_analysis.png
  ✅ 09_genre_cooccurrence.png
  ✅ 10_genre_trends.png
─── Section 6: Studio Analysis ───
  ✅ 11_studio_analysis.png
─── Section 7: Popularity vs Quality ───
  ✅ 12_popularity_vs_quality.png
  ✅ 13_most_popular.png
─── Section 8: Episodes & Duration ───
  ✅ 14_episodes_duration.png
─── Section 9: Source Material ───
  ✅ 15_source_material.png
─── Section 10: Rating (Age) Analysis ───
  ✅ 16_rating_analysis.png
─── Section 11: Top & Bottom Anime ───
  ✅ 17_top25_anime.png
  ✅ 18_bottom25_anime.png
─── Section 12: Hidden Gems & Overrated ──

In [3]:
%pip install pandas matplotlib seaborn wordcloud scipy --break-system-packages


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.patheffects as pe
import seaborn as sns
from scipy import stats
from collections import Counter
from pathlib import Path
import textwrap
import re
import warnings

warnings.filterwarnings("ignore")

# ─────────────────────────────────────────────────────────────────────────────
#  GLOBAL STYLE — Research Paper Aesthetic
# ─────────────────────────────────────────────────────────────────────────────
plt.rcParams.update({
    # Typography
    "font.family":        "serif",
    "font.serif":         ["DejaVu Serif", "Times New Roman", "Georgia"],
    "font.size":          12,
    "axes.titlesize":     15,
    "axes.titleweight":   "bold",
    "axes.labelsize":     13,
    "xtick.labelsize":    11,
    "ytick.labelsize":    11,
    "legend.fontsize":    11,
    "legend.title_fontsize": 12,

    # Figure
    "figure.facecolor":   "white",
    "figure.dpi":         200,
    "savefig.dpi":        300,
    "savefig.bbox":       "tight",
    "savefig.pad_inches": 0.15,
    "figure.figsize":     (10, 6),

    # Axes
    "axes.facecolor":     "#FAFAFA",
    "axes.edgecolor":     "#CCCCCC",
    "axes.linewidth":     0.8,
    "axes.grid":          True,
    "axes.spines.top":    False,
    "axes.spines.right":  False,

    # Grid
    "grid.color":         "#E8E8E8",
    "grid.linewidth":     0.5,
    "grid.alpha":         0.7,

    # Ticks
    "xtick.direction":    "out",
    "ytick.direction":    "out",
    "xtick.major.width":  0.6,
    "ytick.major.width":  0.6,
    "xtick.major.size":   4,
    "ytick.major.size":   4,

    # Lines
    "lines.linewidth":    2.0,
    "lines.antialiased":  True,
    "patch.linewidth":    0.5,
})

# ── Palettes ──
PAL_MAIN      = "#2C5F8A"           # Deep academic blue
PAL_ACCENT    = "#D94F3B"           # Scholarly red
PAL_ACCENT2   = "#3A8A6E"           # Muted teal
PAL_SEQ       = "crest"             # Seaborn sequential
PAL_DIV       = "vlag"              # Seaborn diverging
PAL_CAT       = "deep"              # Seaborn categorical
PAL_MUTED     = sns.color_palette("muted", 10)
PAL_8         = sns.color_palette("Set2", 8)
PAL_GRAD      = sns.color_palette("ch:start=0.2,rot=-0.3", as_cmap=True)
BG_ANNOTATE   = dict(boxstyle="round,pad=0.3", fc="white", ec="#CCCCCC", alpha=0.9)

# ─── Paths ───────────────────────────────────────────────────────────────────
INPUT_CSV = "mal_top_anime.csv"
FIG_DIR = Path("figures")
FIG_DIR.mkdir(exist_ok=True)

saved = []  # (filename, title, caption)


def save(fig, name, title="", caption=""):
    path = FIG_DIR / name
    fig.savefig(path, facecolor="white")
    plt.close(fig)
    saved.append((name, title, caption))
    print(f"  ✓ {name}")


def truncate(text, n=38):
    return textwrap.shorten(str(text), width=n, placeholder="…")


def add_value_labels(ax, fmt=".2f", offset=0.02, fontsize=9, horizontal=True):
    """Add value labels to bar chart."""
    for p in ax.patches:
        if horizontal:
            w = p.get_width()
            if w > 0:
                ax.text(w + offset, p.get_y() + p.get_height() / 2,
                        f"{w:{fmt}}", va="center", fontsize=fontsize, color="#444")
        else:
            h = p.get_height()
            if h > 0:
                ax.text(p.get_x() + p.get_width() / 2, h + offset,
                        f"{h:{fmt}}", ha="center", fontsize=fontsize, color="#444")


# ═════════════════════════════════════════════════════════════════════════════
#  1. LOAD & ENGINEER FEATURES
# ═════════════════════════════════════════════════════════════════════════════
print("📂 Loading data…")
df = pd.read_csv(INPUT_CSV)
N = len(df)
print(f"   {N:,} anime loaded\n")

# ── Derived columns ──
def extract_year(row):
    if pd.notna(row["year"]):
        return int(row["year"])
    m = re.search(r"((?:19|20)\d{2})", str(row.get("aired", "")))
    return int(m.group(1)) if m else np.nan

df["year_clean"] = df.apply(extract_year, axis=1)
df["decade"]     = (df["year_clean"] // 10 * 10).astype("Int64")

def parse_duration(d):
    if pd.isna(d) or d == "Unknown":
        return np.nan
    hrs  = re.search(r"(\d+)\s*hr", str(d))
    mins = re.search(r"(\d+)\s*min", str(d))
    t = 0
    if hrs:  t += int(hrs.group(1)) * 60
    if mins: t += int(mins.group(1))
    return t if t > 0 else np.nan

df["duration_min"] = df["duration"].apply(parse_duration)

def explode_col(col):
    return (df[["mal_id", "score", "members", "year_clean", "decade", col]]
            .dropna(subset=[col])
            .assign(**{col: lambda x: x[col].str.split(", ")})
            .explode(col))

genres_ex  = explode_col("genres")
themes_ex  = explode_col("themes")
studios_ex = explode_col("studios")

scored = df[df["score"].notna()].copy()
yearly = df[df["year_clean"].notna()].copy()
yearly["year_clean"] = yearly["year_clean"].astype(int)

print("🎨 Generating publication-quality figures…\n")

# ═════════════════════════════════════════════════════════════════════════════
#  FIG 1 — Missing Data Profile
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 1: Data Completeness ───")
fig, ax = plt.subplots(figsize=(8, 5))
missing = (df.isnull().mean() * 100).sort_values(ascending=True)
missing = missing[missing > 0]
colors = sns.color_palette("crest_r", len(missing))
bars = ax.barh(missing.index, missing.values, color=colors, edgecolor="white", linewidth=0.5)
ax.set_xlabel("Missing Values (%)")
ax.set_title("Data Completeness Profile")
ax.set_xlim(0, missing.max() * 1.15)
for bar, val in zip(bars, missing.values):
    ax.text(bar.get_width() + 0.4, bar.get_y() + bar.get_height() / 2,
            f"{val:.1f}%", va="center", fontsize=9, color="#555")
ax.axvline(x=0, color="#CCCCCC", linewidth=0.8)
sns.despine(left=True)
fig.tight_layout()
save(fig, "fig01_missing_data.png", "Data Completeness Profile",
     f"Percentage of missing values across {len(df.columns)} features. "
     f"'season' and 'year' columns exhibit the highest missingness (~78%).")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 2 — Score Distribution (KDE + Histogram)
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 2: Score Distribution ───")
fig, ax = plt.subplots(figsize=(10, 5.5))
sns.histplot(scored["score"], bins=70, kde=True, color=PAL_MAIN,
             edgecolor="white", linewidth=0.4, alpha=0.75, stat="density",
             kde_kws={"bw_adjust": 0.8}, ax=ax)
mean_s = scored["score"].mean()
med_s  = scored["score"].median()
ax.axvline(mean_s, color=PAL_ACCENT, ls="--", lw=1.8, zorder=5,
           label=f"Mean = {mean_s:.2f}")
ax.axvline(med_s, color=PAL_ACCENT2, ls="-.", lw=1.8, zorder=5,
           label=f"Median = {med_s:.2f}")
ax.set_xlabel("MAL Score")
ax.set_ylabel("Density")
ax.set_title("Distribution of Anime Scores")
ax.legend(frameon=True, fancybox=False, edgecolor="#CCC", loc="upper left")
skew = scored["score"].skew()
ax.text(0.97, 0.92, f"n = {len(scored):,}\nskew = {skew:.2f}",
        transform=ax.transAxes, ha="right", fontsize=10, bbox=BG_ANNOTATE)
sns.despine()
fig.tight_layout()
save(fig, "fig02_score_distribution.png", "Distribution of Anime Scores",
     f"Left-skewed distribution (skew = {skew:.2f}) with mean {mean_s:.2f} and median {med_s:.2f}, "
     f"indicating a concentration of moderately rated titles.")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 3 — Score by Type (Violin + Box)
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 3: Score by Type ───")
type_order = scored.groupby("type")["score"].median().sort_values(ascending=False).index
fig, ax = plt.subplots(figsize=(10, 6))
sns.violinplot(data=scored, x="score", y="type", order=type_order,
               palette="crest", inner=None, linewidth=0.6, alpha=0.5, ax=ax)
sns.boxplot(data=scored, x="score", y="type", order=type_order,
            width=0.15, palette="crest", fliersize=0, linewidth=0.8,
            boxprops=dict(zorder=3), ax=ax)
ax.set_xlabel("Score")
ax.set_ylabel("")
ax.set_title("Score Distribution by Anime Type")
for i, t in enumerate(type_order):
    n = (scored["type"] == t).sum()
    ax.text(scored["score"].max() + 0.08, i, f"n={n:,}", va="center",
            fontsize=9, color="#777")
sns.despine(left=True)
fig.tight_layout()
save(fig, "fig03_score_by_type.png", "Score Distribution by Anime Type",
     "Violin-box hybrid plot showing score distributions across anime types. "
     "Movies exhibit the widest variance, while TV series cluster around 6–7.")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 4 — Type Composition
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 4: Type Breakdown ───")
type_counts = df["type"].value_counts()
fig, axes = plt.subplots(1, 2, figsize=(12, 5), gridspec_kw={"width_ratios": [1.2, 1]})

colors_d = sns.color_palette("crest", len(type_counts))
wedges, texts, autotexts = axes[0].pie(
    type_counts.values, labels=None, autopct=lambda p: f"{p:.1f}%" if p > 3 else "",
    colors=colors_d, startangle=90, pctdistance=0.78,
    wedgeprops=dict(width=0.45, edgecolor="white", linewidth=1.5))
for at in autotexts:
    at.set_fontsize(10)
    at.set_color("#333")
axes[0].legend(type_counts.index, loc="center left", bbox_to_anchor=(-0.15, 0.5),
               frameon=False, fontsize=10)
axes[0].set_title("Proportion by Type", pad=15)

type_score = scored.groupby("type")["score"].agg(["mean", "median", "std"]).sort_values("mean", ascending=True)
y_pos = range(len(type_score))
axes[1].barh(y_pos, type_score["mean"], xerr=type_score["std"],
             color=sns.color_palette("crest", len(type_score)),
             edgecolor="white", linewidth=0.5, capsize=3, error_kw={"linewidth": 0.8, "color": "#999"})
axes[1].set_yticks(y_pos)
axes[1].set_yticklabels(type_score.index)
axes[1].set_xlabel("Mean Score (± σ)")
axes[1].set_title("Mean Score by Type", pad=15)
axes[1].set_xlim(3, 9)
for i, (idx, row) in enumerate(type_score.iterrows()):
    axes[1].text(row["mean"] + row["std"] + 0.08, i,
                 f'{row["mean"]:.2f}', va="center", fontsize=9, color="#555")
sns.despine(ax=axes[1], left=True)
fig.tight_layout(w_pad=3)
save(fig, "fig04_type_breakdown.png", "Anime Type Composition and Quality",
     f"Left: type distribution (n = {N:,}). Right: mean score with standard deviation error bars.")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 5 — Temporal: Production Volume & Score Trend (Dual Axis)
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 5: Temporal Trends ───")
yr_counts = yearly.groupby("year_clean").size()
yr_score  = yearly[yearly["score"].notna()].groupby("year_clean")["score"].agg(["mean", "std"])
yr_merged = yr_counts.to_frame("count").join(yr_score)

fig, ax1 = plt.subplots(figsize=(12, 5.5))
ax1.fill_between(yr_merged.index, yr_merged["count"], alpha=0.25, color=PAL_MAIN)
ax1.plot(yr_merged.index, yr_merged["count"], color=PAL_MAIN, lw=2, label="Anime Count")
ax1.set_xlabel("Year")
ax1.set_ylabel("Number of Anime Produced", color=PAL_MAIN)
ax1.tick_params(axis="y", labelcolor=PAL_MAIN)

peak_yr = yr_counts.idxmax()
ax1.annotate(f"Peak: {peak_yr}\n({yr_counts.max():,} titles)",
             xy=(peak_yr, yr_counts.max()), xytext=(peak_yr - 12, yr_counts.max() * 0.85),
             arrowprops=dict(arrowstyle="-|>", color="#555", lw=1.2),
             fontsize=10, bbox=BG_ANNOTATE)

ax2 = ax1.twinx()
ax2.plot(yr_merged.index, yr_merged["mean"], color=PAL_ACCENT, lw=2, ls="-", label="Mean Score")
ax2.fill_between(yr_merged.index,
                 yr_merged["mean"] - yr_merged["std"],
                 yr_merged["mean"] + yr_merged["std"],
                 alpha=0.1, color=PAL_ACCENT)
ax2.set_ylabel("Mean Score", color=PAL_ACCENT)
ax2.tick_params(axis="y", labelcolor=PAL_ACCENT)
ax2.set_ylim(3, 9)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper left",
           frameon=True, fancybox=False, edgecolor="#CCC")
ax1.set_title("Anime Production Volume and Average Score Over Time")
ax1.set_xlim(yr_merged.index.min(), yr_merged.index.max())
sns.despine(right=False)
fig.tight_layout()
save(fig, "fig05_temporal_trends.png", "Anime Production Volume and Score Trend",
     f"Dual-axis plot showing production count (blue) and mean score with ±1σ band (red). "
     f"Production peaked in {peak_yr}.")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 6 — Decade Score Distributions (Ridgeline)
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 6: Decade Ridgeline ───")
decade_df = scored[scored["decade"].notna() & (scored["decade"] >= 1970)].copy()
decade_df["decade"] = decade_df["decade"].astype(int)
decades = sorted(decade_df["decade"].unique())

fig, axes = plt.subplots(len(decades), 1, figsize=(10, len(decades) * 0.95 + 1),
                         sharex=True, gridspec_kw={"hspace": -0.4})
colors_ridge = sns.color_palette("crest", len(decades))

for i, (dec, ax) in enumerate(zip(decades, axes)):
    subset = decade_df[decade_df["decade"] == dec]["score"]
    sns.kdeplot(subset, ax=ax, fill=True, alpha=0.7, color=colors_ridge[i],
                linewidth=1.2, bw_adjust=0.8, clip=(1, 10))
    ax.set_ylabel("")
    ax.set_yticks([])
    ax.text(-0.01, 0.3, f"{dec}s", transform=ax.transAxes, fontsize=11,
            fontweight="bold", ha="right", color="#444")
    ax.text(0.97, 0.3, f"n={len(subset):,}  μ={subset.mean():.2f}",
            transform=ax.transAxes, fontsize=9, ha="right", color="#777")
    for spine in ax.spines.values():
        spine.set_visible(False)
    ax.grid(False)
    ax.patch.set_alpha(0)

axes[-1].set_xlabel("Score")
axes[0].set_title("Score Distributions by Decade (Ridgeline Plot)", pad=20)
fig.tight_layout()
save(fig, "fig06_decade_ridgeline.png", "Score Distributions by Decade",
     "Ridgeline (joy) plot showing how the score distribution evolved across decades.")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 7 — Seasonal Patterns
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 7: Seasonal Patterns ───")
seasonal = scored[scored["season"].notna()].copy()
season_order = ["winter", "spring", "summer", "fall"]
season_colors = {"winter": "#5B8DB8", "spring": "#7DB87D", "summer": "#E8A838", "fall": "#C95B4B"}

if len(seasonal) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    season_counts = seasonal["season"].value_counts().reindex(season_order)
    sns.barplot(x=season_counts.index, y=season_counts.values,
                palette=[season_colors[s] for s in season_order],
                edgecolor="white", linewidth=0.8, ax=axes[0])
    axes[0].set_title("Release Count by Season")
    axes[0].set_ylabel("Count")
    axes[0].set_xlabel("")
    for i, v in enumerate(season_counts.values):
        axes[0].text(i, v + 20, f"{v:,}", ha="center", fontsize=10, color="#555")

    sns.violinplot(data=seasonal, x="season", y="score", order=season_order,
                   palette=[season_colors[s] for s in season_order],
                   inner="quartile", linewidth=0.8, ax=axes[1])
    axes[1].set_title("Score Distribution by Season")
    axes[1].set_ylabel("Score")
    axes[1].set_xlabel("")
    means = seasonal.groupby("season")["score"].mean().reindex(season_order)
    for i, (s, m) in enumerate(means.items()):
        axes[1].text(i, 1.8, f"μ={m:.2f}", ha="center", fontsize=9, color="#555")
    axes[1].set_ylim(1, 10)

    for ax in axes:
        sns.despine(ax=ax)
    fig.tight_layout(w_pad=3)
    save(fig, "fig07_seasonal.png", "Seasonal Release Patterns",
         "Left: anime count per season. Right: score violin plots by season of release.")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 8 — Top Genres: Count and Score
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 8: Genre Analysis ───")
genre_counts = genres_ex["genres"].value_counts()
genre_score  = (genres_ex[genres_ex["score"].notna()]
                .groupby("genres")["score"]
                .agg(["mean", "count", "std"])
                .query("count >= 50")
                .sort_values("mean"))

fig, axes = plt.subplots(1, 2, figsize=(13, 8), gridspec_kw={"width_ratios": [1, 1]})
top20_genres = genre_counts.head(20)

sns.barplot(x=top20_genres.values, y=top20_genres.index,
            palette="crest_r", edgecolor="white", linewidth=0.5, ax=axes[0])
axes[0].set_xlabel("Number of Anime")
axes[0].set_ylabel("")
axes[0].set_title("Most Common Genres")
for p in axes[0].patches:
    w = p.get_width()
    axes[0].text(w + 30, p.get_y() + p.get_height() / 2,
                 f"{int(w):,}", va="center", fontsize=9, color="#555")

top20_score = genre_score.tail(20)
sns.barplot(x=top20_score["mean"].values, y=top20_score.index,
            palette="crest", edgecolor="white", linewidth=0.5, ax=axes[1])
axes[1].errorbar(top20_score["mean"].values, range(len(top20_score)),
                 xerr=top20_score["std"].values, fmt="none", ecolor="#999",
                 elinewidth=0.8, capsize=2)
axes[1].set_xlabel("Mean Score (± σ)")
axes[1].set_ylabel("")
axes[1].set_title("Highest-Rated Genres (min. 50 entries)")
axes[1].set_xlim(5, 8.5)
for i, (idx, row) in enumerate(top20_score.iterrows()):
    axes[1].text(row["mean"] + row["std"] + 0.05, i,
                 f'{row["mean"]:.2f} (n={int(row["count"])})',
                 va="center", fontsize=8.5, color="#555")

for ax in axes:
    sns.despine(ax=ax, left=True)
fig.tight_layout(w_pad=3)
save(fig, "fig08_genre_analysis.png", "Genre Frequency and Quality Analysis",
     f"Left: top 20 genres by count. Right: highest-rated genres with ±1σ error bars. "
     f"Most prevalent genre: {genre_counts.index[0]}.")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 9 — Genre Co-occurrence Heatmap
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 9: Genre Co-occurrence ───")
top12_g = genre_counts.head(12).index.tolist()
co = pd.DataFrame(0, index=top12_g, columns=top12_g)
for glist in df.dropna(subset=["genres"])["genres"].str.split(", "):
    filt = [g for g in glist if g in top12_g]
    for i, g1 in enumerate(filt):
        for g2 in filt[i + 1:]:
            co.loc[g1, g2] += 1
            co.loc[g2, g1] += 1

mask = np.triu(np.ones_like(co, dtype=bool), k=0)
fig, ax = plt.subplots(figsize=(9, 7.5))
sns.heatmap(co, mask=mask, annot=True, fmt="d", cmap="crest",
            linewidths=0.8, linecolor="white", square=True,
            cbar_kws={"shrink": 0.75, "label": "Co-occurrence Count"},
            annot_kws={"fontsize": 9}, ax=ax)
ax.set_title("Genre Co-occurrence Matrix (Top 12 Genres)")
ax.tick_params(axis="both", length=0)
fig.tight_layout()
save(fig, "fig09_genre_cooccurrence.png", "Genre Co-occurrence Heatmap",
     "Lower-triangle heatmap of genre pair frequencies among the 12 most common genres.")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 10 — Genre Trends Over Time (Stacked Area)
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 10: Genre Temporal Trends ───")
top6_g = genre_counts.head(6).index.tolist()
gt = (genres_ex[genres_ex["genres"].isin(top6_g) & (genres_ex["year_clean"] >= 1985)]
      .groupby(["year_clean", "genres"]).size().unstack(fill_value=0))
gt_smooth = gt.rolling(3, center=True, min_periods=1).mean()

fig, ax = plt.subplots(figsize=(12, 5.5))
palette_area = sns.color_palette("crest", len(top6_g))
ax.stackplot(gt_smooth.index, *[gt_smooth[g] for g in top6_g],
             labels=top6_g, colors=palette_area, alpha=0.8, linewidth=0)
ax.set_xlabel("Year")
ax.set_ylabel("Number of Titles (3-year rolling avg)")
ax.set_title("Evolution of Top 6 Genres Over Time")
ax.legend(loc="upper left", frameon=True, fancybox=False, edgecolor="#CCC", ncol=3)
ax.set_xlim(gt_smooth.index.min(), gt_smooth.index.max())
sns.despine()
fig.tight_layout()
save(fig, "fig10_genre_trends.png", "Genre Temporal Evolution",
     "Stacked area chart (3-year rolling average) showing the rise of Action and Comedy-tagged titles.")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 11 — Studio Analysis (Lollipop Charts)
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 11: Studio Analysis ───")
studio_counts = studios_ex["studios"].value_counts()
studio_score  = (studios_ex[studios_ex["score"].notna()]
                 .groupby("studios")["score"]
                 .agg(["mean", "count"])
                 .query("count >= 20")
                 .sort_values("mean"))

fig, axes = plt.subplots(1, 2, figsize=(13, 8))

top20_st = studio_counts.head(20)
y_pos = range(len(top20_st))
axes[0].hlines(y=y_pos, xmin=0, xmax=top20_st.values, color=PAL_MAIN, alpha=0.6, linewidth=1.5)
axes[0].scatter(top20_st.values, y_pos, color=PAL_MAIN, s=50, zorder=5, edgecolor="white", linewidth=0.5)
axes[0].set_yticks(y_pos)
axes[0].set_yticklabels(top20_st.index)
axes[0].invert_yaxis()
axes[0].set_xlabel("Number of Anime")
axes[0].set_title("Most Prolific Studios")
for i, v in enumerate(top20_st.values):
    axes[0].text(v + 5, i, f"{v:,}", va="center", fontsize=9, color="#555")

top20_sq = studio_score.tail(20)
y_pos2 = range(len(top20_sq))
axes[1].hlines(y=y_pos2, xmin=5.5, xmax=top20_sq["mean"].values, color=PAL_ACCENT2, alpha=0.6, linewidth=1.5)
axes[1].scatter(top20_sq["mean"].values, y_pos2, color=PAL_ACCENT2, s=50, zorder=5, edgecolor="white", linewidth=0.5)
axes[1].set_yticks(y_pos2)
axes[1].set_yticklabels(top20_sq.index)
axes[1].set_xlabel("Mean Score")
axes[1].set_title("Highest-Rated Studios (min. 20 titles)")
axes[1].set_xlim(5.5, top20_sq["mean"].max() + 0.3)
for i, (idx, row) in enumerate(top20_sq.iterrows()):
    axes[1].text(row["mean"] + 0.03, i,
                 f'{row["mean"]:.2f} (n={int(row["count"])})',
                 va="center", fontsize=8.5, color="#555")

for ax in axes:
    sns.despine(ax=ax, left=True)
    ax.grid(axis="y", visible=False)
fig.tight_layout(w_pad=3)
save(fig, "fig11_studio_analysis.png", "Studio Analysis",
     f"Lollipop charts comparing studio output volume (left) and average rating (right). "
     f"Most prolific: {studio_counts.index[0]}.")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 12 — Popularity vs Quality (Hex + Regression)
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 12: Popularity vs Quality ───")
pq = scored[scored["members"] > 0].copy()
pq["log_members"] = np.log10(pq["members"])

fig, axes = plt.subplots(1, 2, figsize=(13, 5.5))

hb = axes[0].hexbin(pq["log_members"], pq["score"], gridsize=45,
                     cmap="crest", mincnt=1, linewidths=0.1)
cb = fig.colorbar(hb, ax=axes[0], shrink=0.8, label="Count")
slope, intercept, r, p_val, _ = stats.linregress(pq["log_members"], pq["score"])
x_line = np.linspace(pq["log_members"].min(), pq["log_members"].max(), 100)
axes[0].plot(x_line, slope * x_line + intercept, color=PAL_ACCENT, lw=2, ls="--",
             label=f"OLS: r = {r:.3f}, p < 0.001")
axes[0].set_xlabel("log₁₀(Members)")
axes[0].set_ylabel("Score")
axes[0].set_title("Popularity vs Score")
axes[0].legend(loc="lower right", frameon=True, fancybox=False, edgecolor="#CCC", fontsize=10)

fq = scored[(scored["favorites"] > 0)].copy()
fq["log_fav"] = np.log10(fq["favorites"])
hb2 = axes[1].hexbin(fq["log_fav"], fq["score"], gridsize=45,
                      cmap="flare", mincnt=1, linewidths=0.1)
fig.colorbar(hb2, ax=axes[1], shrink=0.8, label="Count")
slope2, intercept2, r2, *_ = stats.linregress(fq["log_fav"], fq["score"])
x2 = np.linspace(fq["log_fav"].min(), fq["log_fav"].max(), 100)
axes[1].plot(x2, slope2 * x2 + intercept2, color="#333", lw=2, ls="--",
             label=f"OLS: r = {r2:.3f}, p < 0.001")
axes[1].set_xlabel("log₁₀(Favorites)")
axes[1].set_ylabel("Score")
axes[1].set_title("Favorites vs Score")
axes[1].legend(loc="lower right", frameon=True, fancybox=False, edgecolor="#CCC", fontsize=10)

for ax in axes:
    sns.despine(ax=ax)
fig.tight_layout(w_pad=3)
save(fig, "fig12_popularity_vs_quality.png", "Popularity–Quality Relationship",
     f"Hexbin density plots with OLS regression. Members–Score: r = {r:.3f}; "
     f"Favorites–Score: r = {r2:.3f}. Both statistically significant (p < 0.001).")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 13 — Top 25 Highest-Rated
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 13: Top 25 Anime ───")
top25 = scored.nlargest(25, "score")[["title", "score", "type", "members"]].reset_index(drop=True)
fig, ax = plt.subplots(figsize=(10, 8))
palette_top = sns.color_palette("crest_r", 25)
y = range(len(top25))
ax.barh(y, top25["score"], color=palette_top, edgecolor="white", linewidth=0.5, height=0.7)
ax.set_yticks(y)
ax.set_yticklabels([truncate(t, 42) for t in top25["title"]], fontsize=9.5)
ax.invert_yaxis()
ax.set_xlabel("Score")
ax.set_xlim(top25["score"].min() - 0.15, top25["score"].max() + 0.15)
ax.set_title("Top 25 Highest-Rated Anime on MyAnimeList")
for i, (_, row) in enumerate(top25.iterrows()):
    ax.text(row["score"] + 0.01, i, f'{row["score"]:.2f}',
            va="center", fontsize=9.5, fontweight="bold", color="#444")
sns.despine(left=True)
ax.grid(axis="y", visible=False)
fig.tight_layout()
save(fig, "fig13_top25.png", "Top 25 Highest-Rated Anime",
     f"Ranked by MAL weighted score. #1: {top25.iloc[0]['title']} ({top25.iloc[0]['score']:.2f}).")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 14 — Most Popular (by Members, color = score)
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 14: Most Popular ───")
top30_pop = df.nlargest(30, "members")[["title", "members", "score"]].reset_index(drop=True)
fig, ax = plt.subplots(figsize=(10, 8))
norm_scores = top30_pop["score"].fillna(0)
colors_pop = sns.color_palette("crest", as_cmap=True)(
    (norm_scores - norm_scores.min()) / (norm_scores.max() - norm_scores.min()))
ax.barh(range(30), top30_pop["members"], color=colors_pop, edgecolor="white", linewidth=0.5, height=0.7)
ax.set_yticks(range(30))
ax.set_yticklabels([truncate(t, 40) for t in top30_pop["title"]], fontsize=9)
ax.invert_yaxis()
ax.set_xlabel("Members")
ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{x / 1e6:.1f}M"))
ax.set_title("Top 30 Most Popular Anime (by Member Count)")
sm = plt.cm.ScalarMappable(cmap="crest",
                           norm=plt.Normalize(norm_scores.min(), norm_scores.max()))
cbar = fig.colorbar(sm, ax=ax, shrink=0.6, pad=0.02)
cbar.set_label("Score", fontsize=11)
sns.despine(left=True)
ax.grid(axis="y", visible=False)
fig.tight_layout()
save(fig, "fig14_most_popular.png", "Most Popular Anime by Membership",
     f"Bar color encodes score. Most popular: {top30_pop.iloc[0]['title']} "
     f"({top30_pop.iloc[0]['members']:,} members).")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 15 — Episode Count vs Score
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 15: Episodes vs Score ───")
eps_df = scored[(scored["episodes"].notna()) & (scored["episodes"] > 0)].copy()
bins = [0, 1, 6, 13, 26, 52, 100, 500, 50000]
labels = ["1", "2–6", "7–13", "14–26", "27–52", "53–100", "101–500", "500+"]
eps_df["ep_bin"] = pd.cut(eps_df["episodes"], bins=bins, labels=labels)

fig, axes = plt.subplots(1, 2, figsize=(13, 5.5))

sns.histplot(eps_df["episodes"].clip(upper=100), bins=50, color=PAL_MAIN,
             edgecolor="white", linewidth=0.3, alpha=0.8, ax=axes[0])
axes[0].axvline(eps_df["episodes"].median(), color=PAL_ACCENT, ls="--", lw=1.5,
                label=f'Median: {eps_df["episodes"].median():.0f}')
axes[0].set_xlabel("Episodes (capped at 100)")
axes[0].set_ylabel("Frequency")
axes[0].set_title("Episode Count Distribution")
axes[0].legend(frameon=True, fancybox=False, edgecolor="#CCC")

sns.boxplot(data=eps_df, x="ep_bin", y="score", order=labels,
            palette="crest", linewidth=0.8, fliersize=1.5, ax=axes[1])
axes[1].set_xlabel("Episode Range")
axes[1].set_ylabel("Score")
axes[1].set_title("Score by Episode Count Bracket")
medians = eps_df.groupby("ep_bin", observed=True)["score"].median().reindex(labels)
for i, (lab, med) in enumerate(medians.items()):
    if pd.notna(med):
        n = (eps_df["ep_bin"] == lab).sum()
        axes[1].text(i, 1.6, f"n={n:,}", ha="center", fontsize=8, color="#777")

for ax in axes:
    sns.despine(ax=ax)
fig.tight_layout(w_pad=3)
save(fig, "fig15_episodes_score.png", "Episode Count Analysis",
     f"Median episode count: {eps_df['episodes'].median():.0f}. "
     "Longer series (27–52 eps) tend to score slightly higher.")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 16 — Source Material (Bubble Lollipop)
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 16: Source Material ───")
src_score = (scored.groupby("source")["score"]
             .agg(["mean", "count", "std"])
             .query("count >= 30")
             .sort_values("mean"))

fig, ax = plt.subplots(figsize=(10, 6))
y_pos = range(len(src_score))
ax.hlines(y=y_pos, xmin=5, xmax=src_score["mean"], color=PAL_MAIN, alpha=0.5, linewidth=1.5)
ax.scatter(src_score["mean"], y_pos, color=PAL_MAIN,
           s=src_score["count"] / src_score["count"].max() * 200 + 30,
           zorder=5, edgecolor="white", linewidth=0.5)
ax.set_yticks(y_pos)
ax.set_yticklabels(src_score.index)
ax.set_xlabel("Mean Score")
ax.set_title("Mean Score by Source Material (bubble size ∝ count)")
ax.set_xlim(5, src_score["mean"].max() + 0.4)
for i, (idx, row) in enumerate(src_score.iterrows()):
    ax.text(row["mean"] + 0.05, i,
            f'{row["mean"]:.2f} (n={int(row["count"]):,})',
            va="center", fontsize=9, color="#555")
sns.despine(left=True)
ax.grid(axis="y", visible=False)
fig.tight_layout()
save(fig, "fig16_source_material.png", "Quality by Source Material",
     "Bubble-lollipop chart; size reflects the number of titles from each source.")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 17 — Rating (Age) Analysis
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 17: Age Rating ───")
rating_df = scored[scored["rating"].notna()].copy()
rating_order = rating_df.groupby("rating")["score"].median().sort_values().index

fig, ax = plt.subplots(figsize=(10, 5.5))
sns.violinplot(data=rating_df, y="rating", x="score", order=rating_order,
               palette="crest", inner="quartile", linewidth=0.8, ax=ax)
ax.set_xlabel("Score")
ax.set_ylabel("")
ax.set_title("Score Distribution by Content Rating")
for i, r in enumerate(rating_order):
    n = (rating_df["rating"] == r).sum()
    ax.text(scored["score"].max() + 0.08, i, f"n={n:,}", va="center", fontsize=9, color="#777")
sns.despine(left=True)
fig.tight_layout()
save(fig, "fig17_age_rating.png", "Score by Content Rating",
     "Violin plots with inner quartile lines. R-rated and PG-13 anime show the broadest score ranges.")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 18 — Hidden Gems vs Overrated
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 18: Hidden Gems ───")
med_members = scored["members"].median()
gems = scored[(scored["score"] >= 8.0) & (scored["members"] < med_members)].nlargest(20, "score")
overrated = scored[(scored["members"] > scored["members"].quantile(0.9)) & (scored["score"] < 7)].nsmallest(20, "score")

fig, axes = plt.subplots(1, 2, figsize=(14, 7))

if len(gems) > 0:
    y = range(len(gems))
    axes[0].hlines(y=y, xmin=7.8, xmax=gems["score"].values, color=PAL_ACCENT2, alpha=0.5, linewidth=1.5)
    axes[0].scatter(gems["score"].values, y, color=PAL_ACCENT2, s=60, zorder=5, edgecolor="white")
    axes[0].set_yticks(y)
    axes[0].set_yticklabels([truncate(t, 35) for t in gems["title"]], fontsize=8.5)
    axes[0].invert_yaxis()
    axes[0].set_xlabel("Score")
    axes[0].set_title("Hidden Gems\n(Score ≥ 8.0, Members < Median)", fontsize=13)
    axes[0].set_xlim(7.8, gems["score"].max() + 0.15)
    for i, (_, row) in enumerate(gems.iterrows()):
        axes[0].text(row["score"] + 0.02, i, f'{row["score"]:.2f}', va="center", fontsize=8.5, color="#555")
    sns.despine(ax=axes[0], left=True)
    axes[0].grid(axis="y", visible=False)

if len(overrated) > 0:
    y2 = range(len(overrated))
    axes[1].hlines(y=y2, xmin=overrated["score"].min() - 0.3, xmax=overrated["score"].values,
                   color=PAL_ACCENT, alpha=0.5, linewidth=1.5)
    axes[1].scatter(overrated["score"].values, y2, color=PAL_ACCENT, s=60, zorder=5, edgecolor="white")
    axes[1].set_yticks(y2)
    axes[1].set_yticklabels([truncate(t, 35) for t in overrated["title"]], fontsize=8.5)
    axes[1].invert_yaxis()
    axes[1].set_xlabel("Score")
    axes[1].set_title("Most Popular, Lowest Rated\n(Top 10% Members, Score < 7.0)", fontsize=13)
    for i, (_, row) in enumerate(overrated.iterrows()):
        axes[1].text(row["score"] + 0.02, i, f'{row["score"]:.2f}', va="center", fontsize=8.5, color="#555")
    sns.despine(ax=axes[1], left=True)
    axes[1].grid(axis="y", visible=False)

fig.tight_layout(w_pad=3)
save(fig, "fig18_hidden_gems.png", "Hidden Gems vs Highly Popular Low-Rated Titles",
     "Left: high-quality anime with below-median popularity. Right: widely-known anime with sub-7 scores.")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 19 — Themes Analysis
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 19: Themes ───")
theme_counts = themes_ex["themes"].value_counts().head(20)
theme_score = (themes_ex[themes_ex["score"].notna()]
               .groupby("themes")["score"]
               .agg(["mean", "count"])
               .query("count >= 30")
               .sort_values("mean").tail(20))

fig, axes = plt.subplots(1, 2, figsize=(13, 7))

sns.barplot(x=theme_counts.values, y=theme_counts.index,
            palette="flare_r", edgecolor="white", linewidth=0.5, ax=axes[0])
axes[0].set_xlabel("Count")
axes[0].set_ylabel("")
axes[0].set_title("Most Common Themes")

sns.barplot(x=theme_score["mean"].values, y=theme_score.index,
            palette="flare", edgecolor="white", linewidth=0.5, ax=axes[1])
axes[1].set_xlabel("Mean Score")
axes[1].set_ylabel("")
axes[1].set_title("Highest-Rated Themes (min. 30 entries)")
axes[1].set_xlim(5.5, 8)

for ax in axes:
    sns.despine(ax=ax, left=True)
fig.tight_layout(w_pad=3)
save(fig, "fig19_themes.png", "Anime Theme Analysis",
     f"Most common theme: {theme_counts.index[0]}. Themes like historical and psychological tend to score higher.")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 20 — Correlation Matrix
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 20: Correlations ───")
num_cols = ["score", "scored_by", "members", "favorites", "episodes", "duration_min"]
corr = scored[num_cols].corr()

fig, ax = plt.subplots(figsize=(8, 6.5))
mask = np.triu(np.ones_like(corr, dtype=bool), k=1)
sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap="vlag", center=0,
            square=True, linewidths=0.8, linecolor="white",
            cbar_kws={"shrink": 0.75, "label": "Pearson r"},
            annot_kws={"fontsize": 11}, ax=ax)
ax.set_title("Correlation Matrix of Numerical Features")
ax.tick_params(axis="both", length=0)
fig.tight_layout()
save(fig, "fig20_correlations.png", "Correlation Matrix",
     "members, scored_by, and favorites are highly inter-correlated. "
     "Score shows moderate positive association with popularity indicators.")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 21 — Status Breakdown
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 21: Status ───")
status_score = scored.groupby("status")["score"].agg(["mean", "median", "count"]).sort_values("mean")

fig, ax = plt.subplots(figsize=(10, 4.5))
palette_status = sns.color_palette("crest", len(status_score))
y = range(len(status_score))
ax.barh(y, status_score["mean"], color=palette_status, edgecolor="white", linewidth=0.5, height=0.6)
ax.scatter(status_score["median"], y, color=PAL_ACCENT, s=40, zorder=5,
           marker="D", edgecolor="white", linewidth=0.5, label="Median")
ax.set_yticks(y)
ax.set_yticklabels(status_score.index)
ax.set_xlabel("Score")
ax.set_title("Mean (bar) and Median (◆) Score by Airing Status")
ax.legend(frameon=True, fancybox=False, edgecolor="#CCC")
for i, (idx, row) in enumerate(status_score.iterrows()):
    ax.text(row["mean"] + 0.03, i,
            f'μ={row["mean"]:.2f}  n={int(row["count"]):,}',
            va="center", fontsize=9, color="#555")
ax.set_xlim(5, 8)
sns.despine(left=True)
ax.grid(axis="y", visible=False)
fig.tight_layout()
save(fig, "fig21_status.png", "Score by Airing Status",
     "Comparison of mean and median scores across airing statuses.")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 22 — Synopsis Word Cloud
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 22: Word Cloud ───")
try:
    from wordcloud import WordCloud
    synopsis_text = " ".join(df["synopsis"].dropna().astype(str))
    stopwords = {"the", "and", "to", "of", "a", "in", "is", "with", "his", "her",
                 "he", "she", "who", "that", "it", "for", "on", "was", "are", "but",
                 "they", "their", "from", "an", "as", "by", "has", "been", "will",
                 "one", "them", "this", "all", "not", "when", "at", "no", "can",
                 "be", "had", "into", "after", "about", "its", "which", "more",
                 "than", "out", "up", "him", "do", "what", "where", "how", "each",
                 "other", "also", "Source", "MAL", "Rewrite", "new", "two"}
    wc = WordCloud(width=1600, height=700, background_color="white",
                   colormap="crest", max_words=120, stopwords=stopwords,
                   prefer_horizontal=0.85, relative_scaling=0.5,
                   min_font_size=8).generate(synopsis_text)
    fig, ax = plt.subplots(figsize=(12, 5.5))
    ax.imshow(wc, interpolation="bilinear")
    ax.axis("off")
    ax.set_title("Most Frequent Terms in Anime Synopses", fontsize=15, fontweight="bold", pad=15)
    fig.tight_layout()
    save(fig, "fig22_wordcloud.png", "Synopsis Word Cloud",
         "Word cloud derived from all non-null synopsis fields, excluding common English stopwords.")
except ImportError:
    print("  ⚠ wordcloud not installed, skipping.")


# ═════════════════════════════════════════════════════════════════════════════
#  FIG 23 — Members Distribution (Log Scale)
# ═════════════════════════════════════════════════════════════════════════════
print("─── Fig 23: Members Distribution ───")
fig, ax = plt.subplots(figsize=(10, 5))
log_members = np.log10(df[df["members"] > 0]["members"])
sns.histplot(log_members, bins=60, kde=True, color=PAL_MAIN,
             edgecolor="white", linewidth=0.3, alpha=0.75, stat="density",
             kde_kws={"bw_adjust": 0.8}, ax=ax)
ax.set_xlabel("log₁₀(Members)")
ax.set_ylabel("Density")
ax.set_title("Distribution of Community Size (Members)")
median_m = df["members"].median()
ax.axvline(np.log10(median_m), color=PAL_ACCENT, ls="--", lw=1.5,
           label=f"Median: {median_m:,.0f}")
ax.legend(frameon=True, fancybox=False, edgecolor="#CCC")
ax.text(0.97, 0.92, f"n = {(df['members'] > 0).sum():,}",
        transform=ax.transAxes, ha="right", fontsize=10, bbox=BG_ANNOTATE)
sns.despine()
fig.tight_layout()
save(fig, "fig23_members_distribution.png", "Community Size Distribution",
     f"Log-normal distribution of member counts. Median: {median_m:,.0f}.")


# ═════════════════════════════════════════════════════════════════════════════
#  HTML REPORT — Clean Academic Style
# ═════════════════════════════════════════════════════════════════════════════
print("\n─── Generating HTML Report ───")

overview_stats = {
    "Total Anime":    f"{N:,}",
    "Scored Anime":   f"{len(scored):,}",
    "Year Range":     f"{int(df['year_clean'].min())}–{int(df['year_clean'].max())}",
    "Mean Score":     f"{scored['score'].mean():.2f}",
    "Median Score":   f"{scored['score'].median():.2f}",
    "Unique Genres":  f"{genres_ex['genres'].nunique()}",
    "Unique Studios": f"{studios_ex['studios'].nunique()}",
    "Total Members":  f"{df['members'].sum() / 1e9:.2f}B",
}

html = ["""<!DOCTYPE html>
<html lang="en"><head>
<meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1">
<title>MAL Anime Analysis Report</title>
<style>
  :root { --bg: #FAFAFA; --text: #333; --accent: #2C5F8A; --border: #E0E0E0;
          --card-bg: #FFF; --subtle: #777; }
  * { margin: 0; padding: 0; box-sizing: border-box; }
  body { font-family: "Georgia", "Times New Roman", serif; background: var(--bg);
         color: var(--text); line-height: 1.7; max-width: 960px; margin: 0 auto; padding: 40px 24px; }
  h1 { font-size: 2em; color: var(--accent); border-bottom: 2px solid var(--accent);
       padding-bottom: 12px; margin-bottom: 8px; }
  h1 small { font-size: 0.5em; font-weight: normal; color: var(--subtle); display: block; margin-top: 4px; }
  .stats { display: grid; grid-template-columns: repeat(auto-fit, minmax(140px, 1fr));
           gap: 12px; margin: 24px 0 36px; }
  .stat { background: var(--card-bg); border: 1px solid var(--border); border-radius: 6px;
          padding: 16px 12px; text-align: center; }
  .stat .val { font-size: 1.5em; font-weight: bold; color: var(--accent); }
  .stat .lbl { font-size: 0.8em; color: var(--subtle); margin-top: 2px; }
  .toc { background: var(--card-bg); border: 1px solid var(--border); border-radius: 6px;
         padding: 20px 24px; margin-bottom: 36px; }
  .toc h2 { font-size: 1.1em; color: var(--accent); margin-bottom: 10px; }
  .toc a { color: var(--accent); text-decoration: none; display: block; padding: 3px 0;
           font-size: 0.95em; }
  .toc a:hover { text-decoration: underline; }
  .figure { background: var(--card-bg); border: 1px solid var(--border); border-radius: 6px;
            margin: 28px 0; overflow: hidden; }
  .figure .caption { padding: 16px 20px 12px; }
  .figure h3 { font-size: 1.1em; color: var(--accent); margin-bottom: 4px; }
  .figure p { font-size: 0.9em; color: var(--subtle); }
  .figure img { width: 100%; display: block; border-top: 1px solid var(--border); }
  .footer { text-align: center; color: var(--subtle); font-size: 0.85em; margin-top: 48px;
            padding-top: 20px; border-top: 1px solid var(--border); }
</style></head><body>
<h1>MyAnimeList — Comprehensive Statistical Analysis<small>""" + f"{N:,} anime · {len(saved)} figures · publication-quality visualizations" + """</small></h1>
<div class="stats">"""]

for lbl, val in overview_stats.items():
    html.append(f'<div class="stat"><div class="val">{val}</div><div class="lbl">{lbl}</div></div>')

html.append('</div><div class="toc"><h2>Table of Contents</h2>')
for i, (fname, title, _) in enumerate(saved, 1):
    anchor = fname.replace(".png", "")
    html.append(f'<a href="#{anchor}">Fig. {i} — {title}</a>')
html.append("</div>")

for i, (fname, title, caption) in enumerate(saved, 1):
    anchor = fname.replace(".png", "")
    html.append(f"""<div class="figure" id="{anchor}">
    <div class="caption"><h3>Figure {i}. {title}</h3><p>{caption}</p></div>
    <img src="{fname}" alt="{title}" loading="lazy"></div>""")

html.append(f"""<div class="footer">
Generated with Python · pandas · matplotlib · seaborn · scipy<br>
Data source: MyAnimeList (via Jikan API v4) · {N:,} entries
</div></body></html>""")

report_path = FIG_DIR / "report.html"
with open(report_path, "w", encoding="utf-8") as f:
    f.write("\n".join(html))

print(f"\n{'═' * 60}")
print(f"  ✅ COMPLETE — {len(saved)} publication-quality figures")
print(f"  📁 {FIG_DIR.resolve()}/")
print(f"  🌐 {report_path.resolve()}")
print(f"{'═' * 60}")

📂 Loading data…
   29,786 anime loaded

🎨 Generating publication-quality figures…

─── Fig 1: Data Completeness ───
  ✓ fig01_missing_data.png
─── Fig 2: Score Distribution ───
  ✓ fig02_score_distribution.png
─── Fig 3: Score by Type ───
  ✓ fig03_score_by_type.png
─── Fig 4: Type Breakdown ───
  ✓ fig04_type_breakdown.png
─── Fig 5: Temporal Trends ───
  ✓ fig05_temporal_trends.png
─── Fig 6: Decade Ridgeline ───
  ✓ fig06_decade_ridgeline.png
─── Fig 7: Seasonal Patterns ───
  ✓ fig07_seasonal.png
─── Fig 8: Genre Analysis ───
  ✓ fig08_genre_analysis.png
─── Fig 9: Genre Co-occurrence ───
  ✓ fig09_genre_cooccurrence.png
─── Fig 10: Genre Temporal Trends ───
  ✓ fig10_genre_trends.png
─── Fig 11: Studio Analysis ───
  ✓ fig11_studio_analysis.png
─── Fig 12: Popularity vs Quality ───
  ✓ fig12_popularity_vs_quality.png
─── Fig 13: Top 25 Anime ───
  ✓ fig13_top25.png
─── Fig 14: Most Popular ───
  ✓ fig14_most_popular.png
─── Fig 15: Episodes vs Score ───
  ✓ fig15_episodes_score.pn